In [1]:
import pandas as pd
import json
import re

In [2]:
with open('SVAMP.json', 'r') as data_file:
    json_data = data_file.read()

In [3]:
train_df = pd.read_json(json_data, orient='records')

In [4]:
statements = []
for i in train_df.Body:
    sent= i.strip().split(".")
    for j in sent:
        j = j.strip()
        if j!="":
            if j[0:2]!="If":
                statements.append(j.lower())
            else:
                statements.append(j[3:].lower())
t_statements =[]
for i in range(len(statements)):
    t_statements.append("".join(e for e in statements[i] if (e.isalpha() or e==' ')))
    t_statements[i] = [re.sub(' +', ' ', t_statements[i]), 0]

In [5]:
questions = []
for i in train_df.Question:
    questions.append(i.lower())
t_questions =[]
for i in questions:
    t_questions.append(["".join(e for e in i if (e.isalpha() or e==' ')), 1])

In [6]:
data = t_statements + t_questions

In [7]:
vocab_data = []
for i in data:
    vocab_data.append(i[0])

In [8]:
df = pd.DataFrame(data, columns=['text','type'])

In [9]:
df = df.sample(frac = 1)

In [10]:
df

,text,type
2687,how many marbles did doug lose,1
765,the second chapter is pages long,0
3128,how many more cakes than pastries did baker sell,1
766,there are different movies and different books...,0
1344,there were parents in the program and pupils too,0
...,...,...
2373,how many more bottles than apple did they have,1
2390,how many bottles of diet soda did they have,1
782,after a typhoon trees in haleys backyard died,0
2147,her room is feet long,0


In [11]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

C:\Users\Naveeta\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Naveeta\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Naveeta\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Naveeta\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarnin

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=0, lowercase=False)
vectorizer.fit(vocab_data)
vectorizer.vocabulary_

{'each': 227,
 'pack': 508,
 'of': 486,
 'dvds': 226,
 'costs': 171,
 'dollars': 218,
 'there': 729,
 'is': 373,
 'discount': 213,
 'on': 492,
 'dan': 187,
 'had': 334,
 'left': 415,
 'with': 822,
 'him': 354,
 'after': 12,
 'he': 343,
 'bought': 94,
 'candy': 124,
 'bar': 60,
 'at': 40,
 'the': 725,
 'start': 683,
 'paco': 512,
 'salty': 620,
 'cookies': 167,
 'and': 25,
 'sweet': 709,
 'ate': 41,
 'children': 144,
 'were': 806,
 'riding': 598,
 'bus': 107,
 'stop': 693,
 'some': 671,
 'got': 312,
 'off': 487,
 'then': 728,
 'while': 813,
 'altogether': 21,
 'in': 365,
 'olivias': 491,
 'wallet': 783,
 'she': 647,
 'collected': 154,
 'more': 459,
 'from': 291,
 'an': 24,
 'atm': 42,
 'visited': 779,
 'supermarket': 707,
 'jerry': 381,
 'action': 4,
 'figures': 264,
 'shelf': 648,
 'his': 356,
 'room': 606,
 'later': 409,
 'added': 7,
 'to': 745,
 'are': 34,
 'total': 753,
 'now': 482,
 'gave': 301,
 'friend': 288,
 'mary': 446,
 'baking': 55,
 'cake': 116,
 'recipe': 585,
 'calls': 11

In [13]:
vectorizer.transform(vocab_data).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [14]:
from sklearn.model_selection import train_test_split
sentences = df['text'].values
y = df['type'].values
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size =  0.25, random_state = 1000)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)
X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)
X_train

<2412x820 sparse matrix of type '<class 'numpy.int64'>'
	with 19661 stored elements in Compressed Sparse Row format>

In [16]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
print("Accuracy:", score)

Accuracy: 0.9987577639751553


In [17]:
from keras.models import Sequential
from keras import layers
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


Using TensorFlow backend.


In [18]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                8210      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 8,221
Trainable params: 8,221
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train, y_train,epochs=100,verbose=False,validation_data=(X_test, y_test),batch_size=10)

Instructions for updating:
Use tf.cast instead.


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Accuracy:",accuracy)